### tensorflow의 장점

* Python API
* Portability: deploy computation to one or more CPUs or GPUs in a desktop, server, or mobile device with a single API
* Flexibility: from Raspberry Pi, Android, Windows, iOS, Linux to server farms
* Visualization (TensorBoard is da bomb)
* Save and restore models, graphs
* Auto-differentiation autodiff (no more taking derivatives by hand. Yay)
* Large community (~300k commits, ~85k repositories)
* Awesome projects already using TensorFlow

### High level APIs on top of TensorFlow

텐서플로에는 여러가지의 high level API를 제공하고 있다. 인기있는 API로는 keras, TFLearn, Sonnet 등이 있다. High level API는 하고자 하는 실험을 간단하게 만들어 줄수 있으며, 복잡한 모델을 단 몇 줄의 코드로 만들 수 있다. 

### tensorflow의 동작방식

먼저 텐서플로의 그래프 계산에 대해 이해할 필요가 있다. 텐서플로는 그래프라는 독특한 방식을 이용하는데, 모든 텐서플로 프로그램은 두가지 방식으로 구성된다.

1. 그래프를 모은다.(assemble a graph)
2. 세션을 이용해서, 그래프 안에 있는 연산을 실행한다.

*단, eager mode에서는 이러한 방식이 바뀌게 된다.

### 텐서(Tensor)란 무엇일까??

텐서란 간단하게 말하면, 데이터 자체라고 생각하면된다. 
텐서는 텐서플로 안에서 n차원의 array 형태로 구현되어있다. 이 텐서에 데이터를 넣고(input), 그래프를 통과시키면서 연산을 하게된다.

In [1]:
import tensorflow as tf

C:\Users\home\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
a = tf.add(3,5)
print(a)

In [ ]:
위 코드를 살펴보면 a는 3,5를 더한 값인 8이 아니고, 3,5를 더하는 연산이라고 정의되어 있다.
그렇다면, a의 값을 얻기 위해서는 어떻게 해야할까? 
Session을 생성해야한다. session을 생성한 후에, 변수를 할당하고, 그래프를 통과시켜 a의 값을 구하게(fetch)된다.

In [ ]:
sess = tf.Session()
print(sess.run(a))
sess.close()

In [ ]:
#세션을 열고 닫는 과정은 다른 방식의 코드로 작성할 수 있다.
with tf.Session() as sess:
    print(sess.run(a))

Session Object에 대해 슬라이드에서는 다음과 같이 정의한다. Operation 객체를 실행하고, 텐서를 구하는 환경이라고 말하고 있다.

'A Session object encapsulates the environment in which Operation objects are executed, and Tensor objects are evaluated.'

'Session will also allocate memory to store the current values of variables.'

In [ ]:
#몇 가지 그래프를 더 만들어 보는 code
x = 2
y = 3
op1 = tf.add(x,y)
op2 = tf.multiply(x,y)
op3 = tf.pow(op1,op2)
with tf.Session() as sess:
    print(sess.run(op3))


In [ ]:
#useless 연산은 Session에서 실행되지 않으므로, 계산 graph만 저장된다.
x = 2
y = 3
add_op = tf.add(x, y)
mul_op = tf.multiply(x, y)
useless = tf.multiply(x, add_op)
pow_op = tf.pow(add_op, mul_op)
with tf.Session() as sess:
    z = sess.run(pow_op) 

In [ ]:
#Session run의 fetches 값은 arrary 형태로 여러개를 넣을 수도 있다.
x = 2
y = 3
add_op = tf.add(x, y)
mul_op = tf.multiply(x, y)
useless = tf.multiply(x, add_op)
pow_op = tf.pow(add_op, mul_op)
with tf.Session() as sess:
    z, not_useless = sess.run([pow_op, useless])

장치 할당에 대한 자세한 사항은 다음 사이트에서 확인 할 수 있다.
[Tensorflow Document](https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/how_tos/using_gpu/)

In [ ]:
#여러 그래프 연산이 있을 경우, 다른 CPU나 GPU로 병렬 연산 처리가 가능하다. 그래프를 연산 장치 별로 할당하는 방법은 다음과 같다.
with tf.device('/cpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='b')
c = tf.multiply(a,b)

# log_device_placement을 True로 설정하여 세션을 만듭니다.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# 실행 op
print(sess.run(c))

GRAPH

여러 개의 세션을 돌리기 위해서는 여러 graph가 필요하다. 기본적으로 default 그래프가 주어지고 이를 사용하게 된다.


In [11]:
#default 그래프가 아닌 새로운 그래프를 추가하려면 다음과 같이 작성하면 된다.
g = tf.Graph()
with g.as_default():
    x = tf.add(3, 5)
sess = tf.Session(graph=g)
with tf.Session(graph=g) as sess:
    print(sess.run(x))

8


In [ ]:
#default graph를 정의하려면 다음과 같이 정의한다.
g = tf.get_default_graph()

In [12]:
#유저가 만든 그래프와 default 그래프를 섞어서 쓰면 안된다.
g1 = tf.get_default_graph()
g2 = tf.Graph()

#add ops to the default graph
with g1.as_default():
    a = tf.constant(3)
    
#add ops to the user created graph
with g2.as_default():
    b = tf.constant(5)

### 그래프를 사용하는 이유?

1. Save computation. Only run subgraphs that lead to the values you want to fetch.
2. Break computation into small, differential pieces to facilitate auto-differentiation
3. Facilitate distributed computation, spread the work across multiple CPUs, GPUs, TPUs, or other devices
4. Many common machine learning models are taught and visualized as directed graphs